In [1]:
%run "./get required data.ipynb"

In [2]:
from datetime import datetime
from variables import app_vars
import pandas as pd

In [3]:
store_name = app_vars.STORE_NAME
transactions = (
    pd
    .read_csv(
        f'./assets/{store_name}_transactions.csv',
        parse_dates=['CreateDate', 'DeleteDate'],
        date_format='%Y-%m-%d %H:%M:%S',
        dtype={'Store': 'category'},
        usecols=['Id', 'CreateDate', 'HeadCount', 'PrivilegeHeadCount', 'Gross', 'Tags', 'DeleteDate', 'SyncInfoId']
    )
    .dropna(how='all')
)

after_start = transactions['CreateDate'] >= app_vars.START_DATE
before_end = transactions['CreateDate'] < app_vars.END_DATE

transactions = transactions[after_start & before_end]

In [4]:
sync_infos = (
    pd
    .read_csv(
        f'./assets/{store_name}_transactions_sync_info.csv',
        usecols=['Id', 'LocalReference', 'ExternalReference'],
        dtype={'ExternalReference': 'int64', 'LocalReference':'int64'}
    )
)

sync_infos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24918 entries, 0 to 24917
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Id                 24918 non-null  int64
 1   ExternalReference  24918 non-null  int64
 2   LocalReference     24918 non-null  int64
dtypes: int64(3)
memory usage: 584.1 KB


In [5]:
discounts = (
    pd
    .read_csv(
        f'./assets/{store_name}_transaction_discounts.csv',
        usecols=['Amount', 'TransactionId', 'Type', 'DeleteDate']
    )
    .where(lambda x : x['DeleteDate'].isna())
)

discounts

,Type,Amount,TransactionId,DeleteDate
0,SeniorDiscount,88.75000,20834,NaN
1,VatExempt,53.25000,20834,NaN
2,SeniorDiscount,341.07143,20835,NaN
3,VatExempt,204.64286,20835,NaN
4,SeniorDiscount,125.44643,20841,NaN
...,...,...,...,...
23775,VatExempt,53.38393,78590,NaN
23776,SeniorDiscount,158.92857,78593,NaN
23777,VatExempt,95.35714,78593,NaN
23778,SeniorDiscount,113.39286,78600,NaN


In [6]:
merged = (
    transactions
    .merge(
        sync_infos,
        how='inner', left_on='SyncInfoId', right_on='Id',
    )
    .dropna(how='all')
    .drop(['Id_y', 'SyncInfoId', 'LocalReference'], axis=1)
    .rename(columns={'Id_x': 'Id'})
)

merged.head()

,Id,HeadCount,PrivilegeHeadCount,Gross,Tags,CreateDate,DeleteDate,ExternalReference
0,33866,2,2,1408.0,#DineIn#SeniorDiscount#VatExempt,2024-01-01 10:57:05,NaT,6437
1,33867,2,1,1295.0,#TakeOut#SeniorDiscount#VatExempt,2024-01-01 11:06:46,NaT,6438
2,33868,1,0,1399.0,#Delivery,2024-01-01 11:25:31,NaT,6439
3,33944,5,2,3040.0,#DineIn#SeniorDiscount#VatExempt,2024-01-01 12:04:29,NaT,6440
4,33945,3,0,1010.0,#DineIn,2024-01-01 12:42:29,NaT,6441


In [9]:
trs = merged
valid = trs[trs['DeleteDate'].isna()]
cancelled = trs[trs['DeleteDate'].isna() == False]

tr_discs = discounts.merge(trs, how='inner', left_on='TransactionId', right_on='Id')
seniors = tr_discs['Type'] == 'SeniorDiscount'
pwds = tr_discs['Type'] == 'Pwd'
priv_discs = tr_discs[seniors | pwds]
opens = tr_discs['Type'] == 'OpenDiscount'
open_discounts = tr_discs[opens]
vat_exes = tr_discs['Type'] == 'VatExempt'
vat_exclusives = tr_discs[vat_exes]

gross = valid['Gross'].sum()
count = valid['Id'].count()
cancelCount = cancelled['Id'].count()
cancelGross = cancelled['Gross'].sum()
priv_disc = priv_discs['Amount'].sum()
open_disc = open_discounts['Amount'].sum()
vat_of_vat_exempt = vat_exclusives['Amount'].sum()
vat_exempt = vat_of_vat_exempt / 0.12
vatable = gross - vat_exempt
vat = vatable * 0.12
trans_start = valid['ExternalReference'].min()
trans_end = valid['ExternalReference'].max()

date_format = '%m/%d/%Y %I:%M %p'
report_date_from = valid['CreateDate'].min()
report_date_to = valid['CreateDate'].max()

with open(f"./outputs/{store_name}-z-report.txt", 'w', encoding='utf-8') as file:
    file.write(f"{'Consolidated Z Report':^42}\n")
    for col in app_vars.header.split('#'):
        file.write(f"{col:^42}\n")

    file.write("\n")
    file.write(f"{'Print Date:':<21}{datetime.today().strftime(date_format):>21}\n")
    file.write(f"{'Report Date From:':<21}{report_date_from.strftime(date_format):>21}\n")
    file.write(f"{'Report Date To:':<21}{report_date_to.strftime(date_format):>21}\n")
    file.write("=" * 42)
    file.write("\n")
    file.write(f"{'Trans Count:':<21}{count:>21}\n")
    file.write(f"{'Void Count:':<21}{cancelCount:>21}\n")
    file.write(f"{'Void Amount:':<21}{cancelGross:>21,.2f}\n")
    file.write(f"{'Change Fund:':<21}{10000:>21,.2f}\n")
    file.write(f"{'Gross Sales:':<21}{gross:>21,.2f}\n")
    file.write(f"{'Priv. Discount:':<21}{priv_disc:>21,.2f}\n")
    file.write(f"{'Other Discount:':<21}{open_disc:>21,.2f}\n")
    file.write(f"{'VAT Amount:':<21}{vat:>21,.2f}\n")
    file.write(f"{'VATABLE:':<21}{vatable:>21,.2f}\n")
    file.write(f"{'VAT Exempt:':<21}{vat_exempt:>21,.2f}\n")
    file.write(f"{'Zero-Rated:':<21}{0.00:>21,.2f}\n")
    file.write(f"{'Trans Start:':<21}{trans_start:>21}\n")
    file.write(f"{'Trans End:':<21}{trans_end:>21}\n")
    file.write(f"{'Reset Counter:':<21}{0:>21}\n")
    file.write(f"{'Checked By:':<21}{'mark':>21}\n")

with open(f'./outputs/{store_name}-z-report.txt', 'r', encoding='utf-8') as f:
    print(f.read())

Formatted output saved!
          Consolidated Z Report           
                YANZI OPC                 
             Gil Fernando Ave             
                San Roque                 
             City of Marikina             
             NCR 2nd district             
      VAT REG TIN:604-619-459-00001       
          POS S/N: G6FN3050032B           
          Min:23072715021599188           
              TEL:8888-5436               

Print Date:            06/07/2025 03:40 PM
Report Date From:      01/01/2024 10:57 AM
Report Date To:        12/31/2024 09:39 PM
Trans Count:                         13483
Void Count:                             64
Void Amount:                     30,024.00
Change Fund:                     10,000.00
Gross Sales:                 22,531,356.00
Priv. Discount:               1,018,104.35
Other Discount:                   2,643.00
VAT Amount:                   2,138,828.63
VATABLE:                     17,823,571.91
VAT Exempt:                  